<a href="https://colab.research.google.com/github/hjkim909/BERT_Practice/blob/master/DistilBERT_for_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
import torch

     |████████████████████████████████| 1.1MB 6.9MB/s 
     |████████████████████████████████| 1.1MB 40.1MB/s 
     |████████████████████████████████| 890kB 40.6MB/s 
     |████████████████████████████████| 3.0MB 41.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=4ff778a4394b8b2eb0c817b61b877c2ff789d51ea3cbb4f735fc7eab00a336da
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import numpy as np

데이터 불러오기

In [ ]:
!wget http://noisy-text.github.io/2017/files/wnut17train.conll


--2020-10-15 03:47:49--  http://noisy-text.github.io/2017/files/wnut17train.conll
Resolving noisy-text.github.io (noisy-text.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to noisy-text.github.io (noisy-text.github.io)|185.199.108.153|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 493781 (482K) [application/octet-stream]
Saving to: ‘wnut17train.conll’

wnut17train.conll   100%[===================>] 482.21K  --.-KB/s    in 0.03s   

2020-10-15 03:47:50 (18.7 MB/s) - ‘wnut17train.conll’ saved [493781/493781]



In [ ]:
from pathlib import Path
import re

def read_wnut(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

texts, tags = read_wnut('wnut17train.conll')

In [ ]:
#tag와 단어가 매칭된 모습
print(texts[0][0:10], tags[0][0:10], sep='\n')

['@paulwalk', 'It', "'s", 'the', 'view', 'from', 'where', 'I', "'m", 'living']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


데이터 전처리

In [ ]:
#train, validation split
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
unique_tags

{'B-corporation',
 'B-creative-work',
 'B-group',
 'B-location',
 'B-person',
 'B-product',
 'I-corporation',
 'I-creative-work',
 'I-group',
 'I-location',
 'I-person',
 'I-product',
 'O'}

In [ ]:
#Distilbert를 통한 encoding
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

* 대부분의 BERT 모델에선 WordPiece tokenizer를 통해서 토큰화 시킴

* 그러나 NER에선 하나의 토큰을 여러 토큰으로 쪼개면 Tag의 수와 불일치하게 됨

* 이를 방지하기 위해 여기서는 split token의 first subtoken에 대해서만 학습

 예시

* '@HuggingFace' -> ['@', 'hugging', 'face']

* 만약 '@HuggingFace'의 index가 3이라면 위 리스트를 [3, -100, -100]으로 변경(-100은 의미 없는 숫자)

In [ ]:
def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [ ]:
#Dataset 생성

class WNUTDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") #모델 생성에 쓸모 없는 부분(이미 위의 encode_tags에 활용)
val_encodings.pop("offset_mapping")
train_dataset = WNUTDataset(train_encodings, train_labels)
val_dataset = WNUTDataset(val_encodings, val_labels)

Pre-trained 모델 불러오기

In [ ]:
from transformers import DistilBertForTokenClassification
model = DistilBertForTokenClassification.from_pretrained('distilbert-base-cased', num_labels=len(unique_tags))

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

Fine tuning

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', return_dict = True)
model.to(device)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [ ]:
optim = AdamW(model.parameters(), lr=5e-5)
model.train()

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        #labels = labels.view(16, -1) 
        outputs = model(input_ids, attention_mask=attention_mask, labels= labels)
        loss, logits = outputs

        
        loss.backward()
        optim.step()


Evaluation

In [ ]:
eval_loader = DataLoader(val_dataset, batch_size=16, shuffle= False)